# Confusion Matrix

In [ ]:

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
        https://stackoverflow.com/a/50386871
    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

# Data Pickling

In [ ]:
#Import statements
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_curve

In [ ]:
with open('../../data/processed/full_set.pkl', 'rb') as f:
    full_set = pickle.load(f)
with open('../../data/processed/train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('../../data/processed/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('../../data/processed/train_set_30.pkl', 'rb') as f:
    train_set_30 = pickle.load(f)
with open('../../data/processed/test_set_30.pkl', 'rb') as f:
    test_set_30 = pickle.load(f)
with open('../../data/processed/train_set_70.pkl', 'rb') as f:
    train_set_70 = pickle.load(f)
with open('../../data/processed/test_set_70.pkl', 'rb') as f:
    test_set_70 = pickle.load(f)

In [ ]:
test_set_70.head()
train_set.head()

In [ ]:
# Split into X, y format 
X_train = train_set.T.iloc[:-1].T
y_train = train_set.T.iloc[-1].T

X_test = test_set.T.iloc[:-1].T
y_test = test_set.T.iloc[-1].T

In [ ]:
y_train.head()

# Linear Classifier

In [ ]:
# I'm a little suspicious about how complicated the linear kernel is - possibly more than we need.
# A multilayer perceptrol with 0 layers is also a linear classifier if we need it.
from sklearn.svm import SVC
linclf = SVC(kernel="linear")
linclf.fit(X_train, y_train) 
linclf.score(X_test, y_test)

# Multilayer Perceptron

### Experiment with various Neural Network parameters: add or remove nodes, layers and connections, vary the learning rate, epochs and momentum, and validation threshold.

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
def get_mlp():
    mlp = MLPClassifier(random_state=42
                        # more non default parameters? E.g. only stochastic gradient descent has been covered in lectures 
                        # also doesn't reach convergence before timeout with current settings
                       )
    return mlp

### Trying it using keras:

In [ ]:
from numpy.random import seed
import random
import tensorflow as tf
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

from keras.wrappers.scikit_learn import KerasClassifier

# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
from keras.models import Sequential
from keras.layers import Dense

def build_keras_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=2303, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit the keras model on the dataset
# model.fit(X_train, y_train, epochs=10, batch_size=100)

# evaluate the keras model
# loss, accuracy = model.evaluate(X_test, y_test)

kcls = KerasClassifier(build_fn=build_keras_model, epochs=10, batch_size=100)

#Note, if you try running this example in an IPython or Jupyter notebook you may get an error.
#The reason is the output progress bars during training. 
#You can easily turn these off by setting verbose=0 in fit() and evaluate() calls



In [ ]:
# https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
from sklearn.model_selection import StratifiedKFold
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=42)
# todo run this
def cross_validate(classifier, X, y):
    '''    
    Given a classifier and training data:
        * Do 10fold CV
        * average the scores
    What this means is for the caller to interpret.
    Returns average result over CV runs 
    '''
    standardising_classifier = make_pipeline(preprocessing.StandardScaler(), classifier)
    cross_val_score(standardising_classifier, X, y, cv=10)
    return np.mean(scores)

# Running An Experiment

In [ ]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline



def run_experiment(classifier, X_train_data, y_train_data, X_test_data, y_test_data):
    '''
    Given a classifier, training data, and test data:
    * Train on the training data
    * Test on the test data
    * display the confusion matrix and other metrics
    '''
    classifier_pipe = make_pipeline(preprocessing.StandardScaler(), classifier)

    classifier_pipe.fit(X_train_data, y_train_data)
    y_pred = classifier_pipe.predict(X_test_data)

    conf_mat = confusion_matrix(y_test_data, y_pred)
    print(classification_report(y_test_data, y_pred))
    plot_confusion_matrix(conf_mat, target_names=y_test_data.unique().sort())


In [ ]:
print("Original split:")
print(f"{train_set['target'].shape[0]} training examples")
mlp_max = get_mlp()
run_experiment(mlp_max, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")


print("Reduced training set:")
print(f"{train_set_30['target'].shape[0]} training examples")
mlp_mid = get_mlp()
run_experiment(mlp_mid, train_set_30.iloc[:,:-1], train_set_30['target'], test_set_30.iloc[:,:-1], test_set_30['target'])
print("------\n")


print("Smallest training set")
print(f"{train_set_70['target'].shape[0]} training examples")
mlp_small = get_mlp()
run_experiment(mlp_small, train_set_70.iloc[:,:-1], train_set_70['target'], test_set_70.iloc[:,:-1], test_set_70['target'])
print("------\n")


... So the results of the previous cell are a bit too good. Let's keep going with the training set reduction:

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# half the remaining training data
def half_remaining_training_data(train_X, train_y, test_X, test_y):
    X_train_tiny, X_test_tiny_tmp, y_train_tiny, y_test_tiny_tmp = train_test_split(train_X, train_y, test_size=0.5, random_state=42)
    
    # Move the new test split into the existing test data
    X_test_tiny = pd.concat([X_test_tiny_tmp, test_X], axis=0)
    y_test_tiny = pd.concat([y_test_tiny_tmp, test_y], axis=0)
                             
    return X_train_tiny, X_test_tiny, y_train_tiny, y_test_tiny



In [ ]:
X_train_90, X_test_90, y_train_90, y_test_90 = half_remaining_training_data(train_set_70.iloc[:,:-1], train_set_70['target'], test_set_70.iloc[:,:-1], test_set_70['target'])
X_train_95, X_test_95, y_train_95, y_test_95 = half_remaining_training_data(X_train_90, y_train_90, X_test_90, y_test_90)

In [ ]:

print("10% training set")
print(f"{y_train_90.shape[0]} training examples")
mlp_tiny = get_mlp()
run_experiment(mlp_tiny, X_train_90, y_train_90, X_test_90, y_test_90)

print("5% training set")
print(f"{y_train_95.shape[0]} training examples")
mlp_tinier = get_mlp()
run_experiment(mlp_tinier, X_train_95, y_train_95, X_test_95, y_test_95)


In [ ]:
print("Keras model, original data split:")
print(f"{train_set['target'].shape[0]} training examples")

run_experiment(kcls, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")



# Visualize

In [ ]:
# https://keras.io/visualization/
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

### Results: Use confusion matrices as well as other metrics (TP Rate, FP Rate, Precision, Recall, F Measure, ROC Area).